In [1]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
import copy
import sympy as sp
import ipysheet
import simplex_algorithm  as sa
import functools
import tableau_input2 as tableau_input

from ipysheet import sheet, cell, column, row, sheet, cell_range
from IPython.display import clear_output, display, Markdown 

In [2]:
#config
M = sp.symbols('M') #mathematical symbol (placeholder for infinite)
max_iterations = 10  # number of iterations before simplex stops
max_real_vars = 10 #max number of real variables
decimal_digits = 2 #number of decimal digits

In [3]:
# observe and on click functions
def create_input_table_with_file(change, display_table_input, display_output, M):
    # clear old output
    display_output.clear_output()
    display_table_input.clear_output()

    # define buttons
    button_simplex_start = widgets.Button(description='Starte Simplex', disabled=True)
    button_check_input = widgets.Button(description='Überprüfe Eingabe')

    # read and execute file to get variable
    uploaded_tableau = read_upload_variables(change)

    # create input table with variable from file
    input_table = sheet(rows=len(uploaded_tableau), columns=len(uploaded_tableau[0]),
                        row_headers=False, column_headers=False)
    cell_table = fill_and_observe_cells(button_simplex_start, input_table, uploaded_tableau)

    #display
    with display_table_input:
        display(Markdown('## Laden des Standardtableaus'))
        display(input_table)
        display(button_check_input)
        display(button_simplex_start)
    
    #obeserve buttons
    button_simplex_start.on_click(functools.partial(start_simplex, input_table=input_table, 
                                                    sum_var= input_table.columns - 3, 
                                                    display_output=display_output, M=M))

    button_check_input.on_click(functools.partial(tableau_input.check_correct_input,
                                                  button_start_algorithm=button_simplex_start,
                                                  cell_table=cell_table,
                                                  input_table=input_table
                                                  )
                                )



def create_input_table_manually(button_create_input_table, dd_count_restr, dd_count_x_vars ,dd_count_s_vars, 
                                dd_count_a_vars ,display_table_input, display_output,M): 
    
    display_table_input.clear_output()
    display_output.clear_output()
    
    button_simplex_start = widgets.Button(description='Starte Simplex', disabled=True)
    button_check_input = widgets.Button(description='Überprüfe Eingabe')
     
    #create input table
    sum_var=int(dd_count_x_vars.value) + int(dd_count_s_vars.value) + int(dd_count_a_vars.value)
    input_table = ipysheet.sheet(rows=2+float(dd_count_restr.value), columns=sum_var+3,  
                                 row_headers=False, column_headers=False)
    input_table, cell_table = tableau_input.prefill_tableau_text_and_colour(input_table = input_table,
                                                                            dd_count_restr = dd_count_restr, 
                                                                            dd_count_x_vars = dd_count_x_vars ,
                                                                            dd_count_s_vars=dd_count_s_vars, 
                                                                            dd_count_a_vars=dd_count_a_vars,
                                                                            button_start_algorithm = button_simplex_start, 
                                                                            M = M, )

    with display_table_input:
        display(Markdown('## Erzeuge das Standardtableau'))
        display(input_table)
        display(button_check_input)
        display(button_simplex_start)
    
    button_check_input.on_click(functools.partial(tableau_input.check_correct_input, 
                                           button_start_algorithm = button_simplex_start, 
                                           cell_table = cell_table,
                                           input_table = input_table
                                          )
                                )
    
    button_simplex_start.on_click(functools.partial(start_simplex, input_table = input_table, 
                                                    sum_var = sum_var, display_output = display_output, M=M))      
                    
        
def start_simplex(button_simplex_start, input_table, sum_var, display_output, M):
    display_output.clear_output()
    pd.set_option('precision', decimal_digits)
    
    #create df of sheet for calculations
    tableau = ipysheet.to_dataframe(input_table)
    tableau = tableau.apply(pd.to_numeric, errors='ignore', downcast='float')
    tableau.columns = range(0, sum_var+3)
    
    #add the rows cj and cj_zj and calculate values
    tableau = tableau_input.add_missing_cj_and_cj_zj_rows(tableau, M)
    sa.calculate_cj_and_cj_zj(tableau)

    #execute simplex algorithm
    tableaus, messages, pivot_elements = sa.simplex_algorithm(tableau, max_iterations, M)
    output_simplex(tableaus, pivot_elements, messages, display_output)

In [4]:
#functions to shorten lines
def create_dropdowns():
    # quantity of restrictions
    dd_count_restr = widgets.Dropdown(
        options=list(map(str, range(0, 6))),
        value='0',
        description='Restriktionen:',
        disabled=False,
    )

    # quantity of real variables
    dd_count_x_vars = widgets.Dropdown(
        options=list(map(str, range(1, max_real_vars))),
        value='1',
        description='echte Variablen:',
        disabled=False,
    )

    # quantity of slack variables
    dd_count_s_vars = widgets.Dropdown(
        options=list(map(str, range(0, 6))),
        value='0',
        description='Schlupfvariablen:',
        disabled=False,
    )

    # quantity of artificial variables
    dd_count_a_vars = widgets.Dropdown(
        options=list(map(str, range(0, 6))),
        value='0',
        description='künstliche Variablen:',
        disabled=False,
    )
    return dd_count_restr, dd_count_x_vars, dd_count_s_vars, dd_count_a_vars

def fill_and_observe_cells(button_simplex_start, input_table, uploaded_tableau):
    #look at every value of the uploaded tableau
    cell_table = [[''] * input_table.columns for i in range(input_table.rows)]
    for row_id in range(input_table.rows):
        for column_id in range(input_table.columns):
            value = uploaded_tableau[row_id][column_id]
            
            #define cell font_weight
            if row_id == 1 or column_id == 1:
                font_weight = 'bold'
            else:
                font_weight = None
            
            #define cell color
            if row_id == 0 and (column_id == 0 or column_id == 1 or column_id == 2) and value != '0':
                cell_color = 'grey'
            elif (row_id == 0 or column_id == 0) and (value == '-M' or value == 0):
                cell_color = 'white'
            elif row_id == 1 or column_id == 1:
                cell_color = 'white'
            else:
                cell_color = 'yellow'
            
            #give value to cell with defined attributes
            cell_table[row_id][column_id] = cell(row_id, column_id, value, background_color=cell_color,
                                                 numeric_format='0.', font_weight=font_weight)
            #observe every cell for changes
            cell_table[row_id][column_id].observe(functools.partial(tableau_input.update_table,
                                                                    input_table=input_table,
                                                                    button_start_algorithm=button_simplex_start))
    return cell_table


def read_upload_variables(change):
    for key in change['new']:
        string = change['new'][key]['content'].decode('utf-8')
        var_to_value = {}
        exec(string, var_to_value)
        uploaded_tableau = var_to_value['tableau']
    return uploaded_tableau


def output_simplex(tableaus, pivot_elements, messages, display_output):
    #display tableaus and the messages to the tableau
    pivot_row_id = 0
    pivot_column_id = 1

    with display_output:
        display(Markdown('## Ergebnis'))

    for tableau_id in range(0, len(tableaus)):
        with display_output:
            display(Markdown('### ' + str(tableau_id) + '.Tableau'))
            display(tableaus[tableau_id].style \
                    .apply(lambda x: ['background: lightblue' if x.name == pivot_elements[tableau_id][pivot_column_id]
                           else '' for i in x]) \
                    .apply(lambda x: ['background: lightblue' if x.name == pivot_elements[tableau_id][pivot_row_id]
                           else '' for i in x], axis=1) \
                    .hide_index() \
                    .hide_columns())   

        for message_id in range(len(messages[tableau_id])):
            with display_output:
                display(widgets.Label(value=messages[tableau_id][message_id]))

In [5]:
# create widgets
display_variable_input = widgets.Output()
display_table_input = widgets.Output()
display_output = widgets.Output()
uploader = widgets.FileUpload(accept='.txt', multiple=False)


dd_count_restr, dd_count_x_vars, dd_count_s_vars, dd_count_a_vars = create_dropdowns()

button_create_input_table = widgets.Button(description='Erzeuge Eingabetableau!')

# manage widget visibility
dd_count_s_vars.layout.visibility = 'hidden'
dd_count_a_vars.layout.visibility = 'hidden'
button_create_input_table.layout.visibility = 'hidden'

# display
display(Markdown('# Simplex Tableau'))
display(Markdown('## Definiere Art/Anzahl der Variablen'))
display(display_variable_input)
display(display_table_input)
display(display_output)

with display_variable_input:
    display(uploader)
    display(Markdown('#### oder'))
    display(dd_count_restr)
    display(dd_count_x_vars)
    display(dd_count_s_vars)
    display(dd_count_a_vars)
    display(button_create_input_table)

# adjust dropdown values by count restriction change
dd_count_restr.observe(functools.partial(tableau_input.on_restr_change,
                                         dd_count_x_vars=dd_count_x_vars,
                                         dd_count_s_vars=dd_count_s_vars,
                                         dd_count_a_vars=dd_count_a_vars,
                                         button_create_input_table=button_create_input_table,
                                         display_variable_input=display_variable_input),
                       names='value'
                       )

# create a table with the values of the dropdowns 
button_create_input_table.on_click(functools.partial(create_input_table_manually,
                                                     dd_count_restr=dd_count_restr,
                                                     dd_count_x_vars=dd_count_x_vars,
                                                     dd_count_s_vars=dd_count_s_vars,
                                                     dd_count_a_vars=dd_count_a_vars,
                                                     display_table_input=display_table_input,
                                                     display_output=display_output,
                                                     M=M
                                                     )
                                   )

# observe Upload-Button for Upload
uploader.observe(functools.partial(create_input_table_with_file, display_table_input=display_table_input,
                                   display_output=display_output, M=M), names='value') 

# Simplex Tableau

## Definiere Art/Anzahl der Variablen

Output()

Output()

Output()